## Download and extract the dataset
---
### Download link expires on December 18, 2019

In [0]:
# !rm -r *
# !wget https://transfer.sh/Lpw1d/dataset.zip
!unzip -q dataset.zip
!ls -l --block-size=M

total 927M
-rw-r--r-- 1 root root 719M Dec  5 06:52 dataset.csv
-rw-r--r-- 1 root root 209M Dec  6 00:25 dataset.zip


In [1]:
# Libraries
import pandas as pd
import numpy as np
import keras
from keras.callbacks import *
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import optimizers
from sklearn.model_selection import train_test_split, KFold

Using TensorFlow backend.


In [0]:
# Load Data
dataset = np.loadtxt(fname="dataset.csv", delimiter=",", skiprows=1)
n_samples = dataset.shape[0]

X = dataset[:, :-1]
y = dataset[:, -1]

# Reshape the array into 28 x 28 pixel
X = X.reshape(n_samples, 28, 28, 1)
# Normalize the data
X = X / 255.0

y = keras.utils.to_categorical(y, 122)

#train-test split
X_train_all, X_test, y_train_all, y_test = train_test_split(X, y, test_size=0.1, random_state=1337, shuffle=True)

# Training LeNet


In [0]:
def get_model(optimizer_name, learning_rate, loss_func_name):
    model = Sequential()

    model.add(Conv2D(32, kernel_size = 3, activation='relu', input_shape = (28, 28, 1)))
    model.add(Conv2D(32, kernel_size = 3, activation='relu'))
    model.add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
    # model.add(Dropout(0.4))

    model.add(Conv2D(64, kernel_size = 3, activation='relu'))
    model.add(Conv2D(64, kernel_size = 3, activation='relu'))
    model.add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
    # model.add(Dropout(0.4))

    model.add(Conv2D(128, kernel_size = 4, activation='relu'))

    model.add(Flatten())
    # model.add(Dropout(0.4))
    model.add(Dense(122, activation='softmax'))

    optimizer = None
    if optimizer_name == "adam":
        optimizer = optimizers.Adam(lr=learning_rate)
    elif optimizer_name == "sgd":
        optimizer = optimizers.SGD(lr=learning_rate)

    model.compile(optimizer=optimizer, loss=loss_func_name, metrics=["accuracy"])

    return model



k = 10
n_epochs = 15
callbacks = [EarlyStopping(monitor="acc", patience=2)]

for optimizer_name in ["adam", "sgd"]:
    for learning_rate in [0.001, 0.01]:
        for batch_size in [32, 64]:

            model_name = optimizer_name + "_" + str(batch_size) + "_" + str(learning_rate)
            print(model_name, end="...   ")
            val_acc_list = []
            kf = KFold(n_splits=k)

            # for train_index, test_index in kf.split(X_train_all):
            #     X_train, X_val = X_train_all[train_index], X_train_all[test_index]
            #     y_train, y_val = y_train_all[train_index], y_train_all[test_index]

            #     model = get_model(optimizer_name, learning_rate)
            #     model.fit(X_train, y_train, epochs=n_epochs, batch_size=batch_size, verbose=0, callbacks=callbacks)
            #     loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
            #     val_acc_list.append(val_acc)

            model = get_model(optimizer_name, learning_rate, "categorical_crossentropy")
            model.fit(X_train_all, y_train_all, epochs=n_epochs, batch_size=batch_size, verbose=1, callbacks=callbacks)
            # loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
            loss, val_acc = model.evaluate(X_train_all, y_train_all, verbose=1)
            val_acc_list.append(val_acc)

            logger = open("log.txt", "a")
            # print(model_name, np.mean(val_acc_list), np.std(val_acc_list), file=logger)
            print(model_name, np.mean(val_acc_list), file=logger)
            print("mean_val_acc:", np.mean(val_acc_list), "  std_deviation:", np.std(val_acc_list))
            logger.close()

In [0]:
model = get_model("adam", 0.001)
model.fit(X_train_all, y_train_all, epochs=20, batch_size=64, verbose=1, callbacks=callbacks)
model.evaluate(X_test, y_test, verbose = 1)

In [0]:
# from google.colab import files
# files.download('log.txt') 

In [5]:
def get_model(optimizer_name, learning_rate, loss_func_name):
    model = Sequential()

    model.add(Conv2D(32, kernel_size = 3, activation='relu', input_shape = (28, 28, 1)))
    model.add(Conv2D(32, kernel_size = 3, activation='relu'))
    model.add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model.add(Dropout(0.4))

    model.add(Conv2D(64, kernel_size = 3, activation='relu'))
    model.add(Conv2D(64, kernel_size = 3, activation='relu'))
    model.add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model.add(Dropout(0.4))

    model.add(Conv2D(128, kernel_size = 4, activation='relu'))

    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(122, activation='softmax'))

    optimizer = None
    if optimizer_name == "adam":
        optimizer = optimizers.Adam(lr=learning_rate)
    elif optimizer_name == "sgd":
        optimizer = optimizers.SGD(lr=learning_rate)

    model.compile(optimizer=optimizer, loss=loss_func_name, metrics=["accuracy"])

    return model

model = get_model("adam", 0.001, "categorical_crossentropy")
model.summary()


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 12, 12, 32)        25632     
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 10, 10, 64)        18496     
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 8, 8, 64)          36928     
____